In [2]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 867.2 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 4.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

from langchain.vectorstores import Chroma
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

In [4]:
# Load environment variables (for OpenAI API key)
load_dotenv()
openai_api_key = os.environ["OPENAI_KEY"]

# Load your text file
loader = TextLoader("movies.txt")
documents = loader.load()

In [5]:
# Split text into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,        # size of each text chunk
    chunk_overlap=50,      # overlap between chunks to retain context
    separators=["\n", ". ", " "]  # splits text at newlines, sentences, or spaces
)

# Split the loaded document
texts = text_splitter.split_documents(documents)
print(f"Total chunks created: {len(texts)}")
print(f"First chunk: {texts[0].page_content[:200]}...")  # Display first 200 characters of the first chunk


Total chunks created: 21
First chunk: Movie Title: The Shawshank Redemption (1994)
Movie Code: MOV-001

Description:...


In [6]:
# Initialize embeddings (e.g., OpenAI)
embeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key,
    model="text-embedding-3-small"
)

# Store embeddings in vector database (Chroma in-memory DB)
vectordb = Chroma.from_documents(texts, embeddings)

/tmp/ipykernel_542398/1996856946.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


In [7]:
# Create the retriever (fetches relevant document chunks)
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # retrieve top 3 chunks

# Setup Retrieval-Augmented Generation (RAG) model
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(
        api_key=openai_api_key,
        model_name="gpt-4o",
        temperature=0.3  # Set to 0 for deterministic responses),
    ),
    retriever=retriever,
    return_source_documents=True  # Return source documents with answers
)

In [9]:
# Example Query
query = "¿Cuándo se estrenó la primera película de Star Wars?"
response = qa_chain.invoke(query)

print("Answer:", response)

Answer: {'query': '¿Cuándo se estrenó la primera película de Star Wars?', 'result': 'La primera película de Star Wars, conocida como "Star Wars: Episode IV - A New Hope", se estrenó el 25 de mayo de 1977.', 'source_documents': [Document(metadata={'source': 'movies.txt'}, page_content='. Visually stunning, scientifically imaginative, and emotionally powerful, Interstellar is an ambitious masterpiece that deeply contemplates humanity’s place in the universe and the enduring bonds between loved ones.'), Document(metadata={'source': 'movies.txt'}, page_content='Actors:\n\t•\tLeonardo DiCaprio (Dom Cobb)\n\t•\tJoseph Gordon-Levitt (Arthur)\n\t•\tEllen Page (Ariadne)\n\t•\tTom Hardy (Eames)\n\t•\tKen Watanabe (Saito)\n\n---\n\nMovie Title: Interstellar (2014)\nMovie Code: MOV-004\n\nDescription:'), Document(metadata={'source': 'movies.txt'}, page_content='. With masterful direction and unforgettable performances, the film explores themes of family, power, loyalty, and corruption. It’s univer